In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from config import epicorConnectionString

def highlight_blue(s):
    return 'background-color: lightblue'
def highlight_yellow(s):
    return 'background-color: lightyellow'

## 3582
#### Precept Assemlbies

In [41]:
so3582_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3582_query = f.read().split(';')[0].format(SO=3582)

with create_engine(epicorConnectionString).connect() as con:
    so3582_df = pd.read_sql(text(so3582_query), con)

so3582_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3582.csv", columns=so3582_df.columns, index=False, date_format=None)
so3582_df

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
0,3582-01-1,NV10468,1563206,Precept Solid Ring Threaded,30.0,1099.0,Clip/Split Ring,1000.0,34.0,0.928116
1,3582-01-1,NV10778,1553275,5.5 Precept Load Ring,30.0,1100.0,Load Ring,1000.0,34.0,0.927273
2,3582-01-1,NV10937,1553262,5.5mm Precept Polyaxial Tulip,30.0,550.0,Tulip,500.0,29.0,1.581818
3,3582-01-1,NV11871,1557480,6.5 x 50mm Precept Poly SR Shank,30.0,220.0,Shank,200.0,20.0,2.727273
4,3582-01-2,NV11085,1563206,Precept Solid Ring Threaded,83.0,1100.0,Clip/Split Ring,1000.0,34.0,2.565455
...,...,...,...,...,...,...,...,...,...,...
67,3582-15-1,NV15297,1590480,7.5 x 50mm Precept Screw Shank,139.0,219.0,Shank,180.0,20.0,12.694064
68,3582-16-1,NV14649,1582023,Precept Poly C.R. Load Ring,52.0,550.0,Load Ring,500.0,29.0,2.741818
69,3582-16-1,NV14657,1582022,Precept Poly Clip Ring,52.0,1637.0,Clip/Split Ring,1000.0,34.0,1.080024
70,3582-16-1,NV14670,1582025,Precept Poly C.R. Tulip,52.0,650.0,Tulip,500.0,29.0,2.320000


## 3574
#### Reline Assemblies

*Due to a transaction error, the extensions and tulips in 3574-21-1 were assigned twice. We are dropping the duplicate columns before exporting the csv*


In [4]:
so3574_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3574_query = f.read().split(';')[0].format(SO=3574)

with create_engine(epicorConnectionString).connect() as con:
    so3574_df = pd.read_sql(text(so3574_query), con)



so3574_df[so3574_df['JobNum'].str.contains('3574-21-1')].style.applymap(highlight_yellow, pd.IndexSlice[118:121,['PhaseID','JobNum']])

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
115,3574-21-1,NV14678,1642661,Reline Poly Load Ring,92.000000,1098.000000,Load Ring,1000.000000,34.000000,2.848816
116,3574-21-1,NV14682,1642666,Reline Clip Ring Flat,92.000000,1102.000000,Clip/Split Ring,1000.000000,34.000000,2.838475
117,3574-21-1,NV14710,1965XXX,6.5 x 35mm Polyaxial Reline-One Shank,92.000000,174.000000,Shank,165.000000,20.000000,10.574713
118,3574-21-1,NV14733,1642646,Reline Breakoff Extension,92.000000,275.000000,Extension,250.000000,20.000000,6.690909
119,3574-21-1,NV14734,1642646,Reline Breakoff Extension,92.000000,275.000000,Extension,250.000000,20.000000,6.690909
120,3574-21-1,NV14783,1642652,Reline MAS Reduction Tulip Base,92.000000,275.000000,Tulip,250.000000,20.000000,6.690909
121,3574-21-1,NV14794,1642652,Reline MAS Reduction Tulip Base,92.000000,275.000000,Tulip,250.000000,20.000000,6.690909


In [33]:
# Drop the duplicate rows before exporting to csv
so3574_df.drop(labels=[119,121], axis=0, inplace=True)


so3574_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3574.csv", columns=so3574_df.columns, index=False, date_format=None)

# Double check that this line item no longer contains any duplicates
so3574_df[so3574_df['JobNum'].str.contains('3574-21-1')]

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
115,3574-21-1,NV14678,1642661,Reline Poly Load Ring,92.0,1098.0,Load Ring,1000.0,34.0,2.848816
116,3574-21-1,NV14682,1642666,Reline Clip Ring Flat,92.0,1102.0,Clip/Split Ring,1000.0,34.0,2.838475
117,3574-21-1,NV14710,1965XXX,6.5 x 35mm Polyaxial Reline-One Shank,92.0,174.0,Shank,165.0,20.0,10.574713
118,3574-21-1,NV14733,1642646,Reline Breakoff Extension,92.0,275.0,Extension,250.0,20.0,6.690909
120,3574-21-1,NV14783,1642652,Reline MAS Reduction Tulip Base,92.0,275.0,Tulip,250.0,20.0,6.690909


## 3563
#### Reline-O Assemblies

*An extra qty of 7 load rings were assigned to 3563-05-1, possible because the original qty lost some parts or were damaged. We're dropping the 7 qty row here to keep quantites consistent between phaseIDs*

*In addition, we have a couple of line items for MOD assemlies that don't use shanks, they're not subject to this analysis, so we are dropping these rows as well*

In [2]:
so3563_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3563_query = f.read().split(';')[0].format(SO=3563)

with create_engine(epicorConnectionString).connect() as con:
    so3563_df = pd.read_sql(text(so3563_query), con)

    
# 3563-05-1 has an extra qty of 7 load rings that doesnt belong
so3563_df[so3563_df['JobNum'].str.contains('3563-05-1')].style.applymap(highlight_yellow, pd.IndexSlice[['JobNum','TranQty','PhaseID']])

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
16,3563-05-1,NV12044,1642664,Reline Closed Iliac Load Ring,7.000000,550.000000,Load Ring,500.000000,29.000000,0.369091
17,3563-05-1,NV12044,1642664,Reline Closed Iliac Load Ring,30.000000,550.000000,Load Ring,500.000000,29.000000,1.581818
18,3563-05-1,NV12374,1642657,Reline Closed Iliac Tulip,30.000000,550.000000,Tulip,500.000000,29.000000,1.581818
19,3563-05-1,NV12883,1759000,9.5 x 60mm Grit Blast Solid Canccellous Shank,30.000000,88.000000,Shank,25.000000,13.000000,4.431818
20,3563-05-1,NV14395,1642666,Reline Clip Ring Flat,30.000000,1099.000000,Clip/Split Ring,1000.000000,34.000000,0.928116


In [63]:

cond1 = so3563_df['JobNum'].str.contains('3563-07-1')
cond2 = so3563_df['JobNum'].str.contains('3563-08-1')
cond3 = so3563_df['JobNum'].str.contains('3563-08-2')

so3563_df[cond1 | cond2 | cond3].style.applymap(highlight_yellow, pd.IndexSlice['PhaseID'])

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
29,3563-07-1,NV14534,1621666,Reline Open Reduction MOD Tulip,208.000000,263.000000,Tulip,250.000000,20.000000,15.817490
30,3563-07-1,NV14548,1610652,Reline MOD Split Ring,208.000000,2204.000000,Clip/Split Ring,2000.000000,42.000000,3.963702
31,3563-07-1,NV14572,1642799,Reline MOD Load Ring,208.000000,1100.000000,Load Ring,1000.000000,34.000000,6.429091
32,3563-08-1,NV12056,1621666,Reline Open Reduction MOD Tulip,20.000000,315.000000,Tulip,300.000000,29.000000,1.841270
33,3563-08-1,NV12955,1610652,Reline MOD Split Ring,20.000000,1100.000000,Clip/Split Ring,1000.000000,34.000000,0.618182
34,3563-08-1,NV14572,1642799,Reline MOD Load Ring,20.000000,1100.000000,Load Ring,1000.000000,34.000000,0.618182
35,3563-08-2,NV14033,1610652,Reline MOD Split Ring,72.000000,1098.000000,Clip/Split Ring,1000.000000,34.000000,2.229508
36,3563-08-2,NV14572,1642799,Reline MOD Load Ring,65.000000,1100.000000,Load Ring,1000.000000,34.000000,2.009091
37,3563-08-2,NV14618,1621666,Reline Open Reduction MOD Tulip,60.000000,277.000000,Tulip,250.000000,20.000000,4.332130


In [11]:
so3563_df.drop(labels=[16], axis=0, inplace=True)
so3563_df.drop(so3563_df[cond1 | cond2 | cond3].index, axis=0, inplace=True)

so3563_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3563.csv", columns=so3563_df.columns, index=False, date_format=None)


c:\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


## 3561
#### Armada and Precept Assemblies

In [5]:
so3561_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3561_query = f.read().split(';')[0].format(SO=3561)

with create_engine(epicorConnectionString).connect() as con:
    so3561_df = pd.read_sql(text(so3561_query), con)
    
so3561_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3561.csv", columns=so3561_df.columns, index=False, date_format=None)

so3561_df

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
0,3561-01-1,NV10200,1541900,5.5 Poly Tulip 5.5 Ti,27.0,549.0,Tulip,500.0,29.0,1.426230
1,3561-01-1,NV10458,1541901,5.5mm Load Ring Ti,27.0,550.0,Load Ring,500.0,29.0,1.423636
2,3561-01-1,NV14629,1541902,"Deformity Poly Split Ring, Ti",27.0,604.0,Clip/Split Ring,500.0,29.0,1.296358
3,3561-01-1,NV14639,1567720,6.5 x 45mm Fenestrated Shank,27.0,55.0,Shank,25.0,13.0,6.381818
4,3561-02-1,NV10200,1541900,5.5 Poly Tulip 5.5 Ti,27.0,549.0,Tulip,500.0,29.0,1.426230
...,...,...,...,...,...,...,...,...,...,...
92,3561-15-1,nv15275,1582023,Precept Poly C.R. Load Ring,72.0,551.0,Load Ring,500.0,29.0,3.789474
93,3561-16-1,NV12110,1582025,Precept Poly C.R. Tulip,34.0,550.0,Tulip,500.0,29.0,1.792727
94,3561-16-1,NV12323,1590480,8.5 x 40mm Precept Screw Shank,28.0,28.0,Shank,25.0,13.0,13.000000
95,3561-16-1,NV14211,1582022,Precept Poly Clip Ring,28.0,1320.0,Clip/Split Ring,1000.0,34.0,0.721212


## 3555
#### Reline Assemblies

In [7]:
so3555_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3555_query = f.read().split(';')[0].format(SO=3555)

with create_engine(epicorConnectionString).connect() as con:
    so3555_df = pd.read_sql(text(so3555_query), con)
    
so3555_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3555.csv", columns=so3555_df.columns, index=False, date_format=None)

so3555_df[0:10]

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
0,3555-01-1,NV14415,1642652,Reline MAS Reduction Tulip Base,80.0,275.0,Tulip,250.0,20.0,5.818182
1,3555-01-1,NV14422,1642646,Reline Breakoff Extension,80.0,275.0,Extension,250.0,20.0,5.818182
2,3555-01-1,nv14434,1642666,Reline Clip Ring Flat,80.0,1308.0,Clip/Split Ring,1000.0,34.0,2.079511
3,3555-01-1,NV14680,1642661,Reline Poly Load Ring,80.0,1478.0,Load Ring,1000.0,34.0,1.840325
4,3555-01-1,nv14855,1751200,7.5 x 55mm Cannulated Cancellous Shank,80.0,282.0,Shank,250.0,20.0,5.673759
5,3555-02-1,NV14383,1642652,Reline MAS Reduction Tulip Base,20.0,275.0,Tulip,250.0,20.0,1.454545
6,3555-02-1,NV14389,1642646,Reline Breakoff Extension,20.0,275.0,Extension,250.0,20.0,1.454545
7,3555-02-1,NV14391,1642661,Reline Poly Load Ring,20.0,1100.0,Load Ring,1000.0,34.0,0.618182
8,3555-02-1,NV14395,1642666,Reline Clip Ring Flat,20.0,1099.0,Clip/Split Ring,1000.0,34.0,0.618744
9,3555-02-1,NV7742,1751200,4.5 x 55mm Cannulated Cancellous Shank,22.0,22.0,Shank,40.0,13.0,13.000000


## 3500
#### RSS Assemblies

In [9]:
so3500_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3500_query = f.read().split(';')[0].format(SO=3500)

with create_engine(epicorConnectionString).connect() as con:
    so3500_df = pd.read_sql(text(so3500_query), con)
    
so3500_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3500.csv", columns=so3500_df.columns, index=False, date_format=None)

so3500_df[0:10]

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
0,3500-01-1,NV11907,1722000,4.0 x 20mm .265 Solid Cancellous Shank,17.0,176.0,Shank,160.0,20.0,1.931818
1,3500-01-1,NV14170,1773781,RSS Clip Ring Formed,17.0,1096.0,Clip/Split Ring,470.0,29.0,0.449818
2,3500-01-1,NV14172,1773788,RSS Load Ring,17.0,1120.0,Load Ring,400.0,29.0,0.440179
3,3500-01-1,NV14201,1773780,RSS Polyaxial Screw Tulip,17.0,548.0,Tulip,500.0,29.0,0.899635
4,3500-01-2,NV11906,1722000,4.0 x 20mm .265 Solid Cancellous Shank,12.0,174.0,Shank,160.0,20.0,1.379310
5,3500-01-2,NV14172,1773788,RSS Load Ring,12.0,1120.0,Load Ring,400.0,29.0,0.310714
6,3500-01-2,NV14201,1773780,RSS Polyaxial Screw Tulip,12.0,548.0,Tulip,500.0,29.0,0.635036
7,3500-01-2,NV14615,1773781,RSS Clip Ring Formed,12.0,1095.0,Clip/Split Ring,1000.0,34.0,0.372603
8,3500-02-1,NV13539,1722000,4.5 x 25mm .265 Solid Cancellous Shank,75.0,275.0,Shank,250.0,20.0,5.454545
9,3500-02-1,NV14169,1773781,RSS Clip Ring Formed,75.0,1098.0,Clip/Split Ring,1000.0,34.0,2.322404


## 3497
#### Reline Assemblies

*So line 1-4 and 4-1 aren't correctly pulling data for the tulips and extensions because their weld jobs were mistakenly labeled*
*as 'Assy' instead of 'Sub-assy' PhaseID. Its too much to just work around with our query, so we'll just drop the results*

In [10]:
so3497_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3497_query = f.read().split(';')[0].format(SO=3497)

with create_engine(epicorConnectionString).connect() as con:
    so3497_df = pd.read_sql(text(so3497_query), con)
    

cond1 = so3497_df['JobNum'].str.contains('3497-01-1')
cond2 = so3497_df['JobNum'].str.contains('3497-01-4')
cond3 = cond1 | cond2
so3497_df[(cond1 | cond2)].style.applymap(highlight_yellow, pd.IndexSlice[['PhaseID', 'JobNum']])

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
0,3497-01-1,NV13223,1751200,5.5 x 50mm Cannulated Cancellous Shank,88.000000,88.000000,Shank,25.000000,13.000000,13.000000
1,3497-01-1,NV14391,1642661,Reline Poly Load Ring,88.000000,1100.000000,Load Ring,1000.000000,34.000000,2.720000
2,3497-01-1,NV14394,1642666,Reline Clip Ring Flat,88.000000,1100.000000,Clip/Split Ring,1000.000000,34.000000,2.720000
13,3497-01-4,NV13360,1751200,5.5 x 50mm Cannulated Cancellous Shank,18.000000,275.000000,Shank,250.000000,20.000000,1.309091
14,3497-01-4,NV14174,1642661,Reline Poly Load Ring,18.000000,1100.000000,Load Ring,1000.000000,34.000000,0.556364
15,3497-01-4,NV14395,1642666,Reline Clip Ring Flat,18.000000,1099.000000,Clip/Split Ring,1000.000000,34.000000,0.556870


In [11]:
so3497_df = so3497_df[~(cond1 | cond2)]

so3497_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3497.csv", columns=so3497_df.columns, index=False, date_format=None)

so3497_df.iloc[0:15]

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
3,3497-01-2,NV13007,1751200,5.5 x 50mm Cannulated Cancellous Shank,40.0,68.0,Shank,25.0,13.0,7.647059
4,3497-01-2,NV13687,1642646,Reline Breakoff Extension,40.0,275.0,Extension,250.0,20.0,2.909091
5,3497-01-2,NV13727,1642652,Reline MAS Reduction Tulip Base,40.0,274.0,Tulip,250.0,20.0,2.919708
6,3497-01-2,NV14174,1642661,Reline Poly Load Ring,40.0,1100.0,Load Ring,1000.0,34.0,1.236364
7,3497-01-2,NV14394,1642666,Reline Clip Ring Flat,44.0,1100.0,Clip/Split Ring,1000.0,34.0,1.360000
8,3497-01-3,NV13633,1751200,5.5 x 50mm Cannulated Cancellous Shank,20.0,275.0,Shank,250.0,20.0,1.454545
9,3497-01-3,NV13873,1642652,Reline MAS Reduction Tulip Base,19.0,275.0,Tulip,250.0,20.0,1.381818
10,3497-01-3,NV14106,1642646,Reline Breakoff Extension,19.0,275.0,Extension,250.0,20.0,1.381818
11,3497-01-3,NV14174,1642661,Reline Poly Load Ring,20.0,1100.0,Load Ring,1000.0,34.0,0.618182
12,3497-01-3,NV14395,1642666,Reline Clip Ring Flat,19.0,1099.0,Clip/Split Ring,1000.0,34.0,0.587807


## 3492
#### Armada and Precept Assemblies

In [12]:
so3492_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments.sql')) as f:
    so3492_query = f.read().split(';')[0].format(SO=3492)

with create_engine(epicorConnectionString).connect() as con:
    so3492_df = pd.read_sql(text(so3492_query), con)
    
so3492_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3492.csv", columns=so3492_df.columns, index=False, date_format=None)

so3492_df

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
0,3492-01-1,NV10200,1541900,5.5 Poly Tulip 5.5 Ti,27.0,549.0,Tulip,500.0,29.0,1.426230
1,3492-01-1,NV10458,1541901,5.5mm Load Ring Ti,27.0,550.0,Load Ring,500.0,29.0,1.423636
2,3492-01-1,NV14430,1567720,6.5 x 50mm Fenestrated Shank,27.0,83.0,Shank,25.0,13.0,4.228916
3,3492-01-1,NV14629,1541902,"Deformity Poly Split Ring, Ti",27.0,604.0,Clip/Split Ring,500.0,29.0,1.296358
4,3492-02-1,NV10937,1553262,5.5mm Precept Polyaxial Tulip,220.0,550.0,Tulip,500.0,29.0,11.600000
...,...,...,...,...,...,...,...,...,...,...
62,3492-13-1,NV14435,1582023,Precept Poly C.R. Load Ring,28.0,550.0,Load Ring,500.0,29.0,1.476364
63,3492-13-2,NV12108,1582025,Precept Poly C.R. Tulip,110.0,550.0,Tulip,500.0,29.0,5.800000
64,3492-13-2,NV14211,1582022,Precept Poly Clip Ring,110.0,1320.0,Clip/Split Ring,1000.0,34.0,2.833333
65,3492-13-2,NV14399,1590480,7.5 x 55mm Precept Screw Shank,110.0,110.0,Shank,95.0,13.0,13.000000


## 3663
#### RSS Screw Assemblies



In [12]:
so3663_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments2.sql')) as f:
    so3663_query = f.read().split(';')[0].format(SO=3663)

with create_engine(epicorConnectionString).connect() as con:
    so3663_df = pd.read_sql(text(so3663_query), con)

    
# Some Clip Rings are mistakenly labeled as Cams, so just correcting those
so3663_df[so3663_df['PhaseID'].str.contains('Cam')].style.applymap(highlight_yellow, pd.IndexSlice[['JobNum','TranQty','PhaseID']])

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
13,003663-12-1,NV15327,1773781,RSS Clip Ring Formed,27.000000,1331.000000,Cam???,1000.000000,34.000000,0.689707
20,003663-14-1,NV15327,1773781,RSS Clip Ring Formed,80.000000,1331.000000,Cam???,1000.000000,34.000000,2.043576
30,003663-16-1,nv15329,1773781,RSS Clip Ring Formed,295.000000,699.000000,Cam???,1000.000000,34.000000,14.349070
32,003663-17-1,nv15327,1773781,RSS Clip Ring Formed,85.000000,1331.000000,Cam???,1000.000000,34.000000,2.171300
37,003663-18-1,nv15327,1773781,RSS Clip Ring Formed,21.000000,1331.000000,Cam???,1000.000000,34.000000,0.536439
41,003663-19-1,nv15327,1773781,RSS Clip Ring Formed,27.000000,1331.000000,Cam???,1000.000000,34.000000,0.689707
44,003663-20-1,NV15327,1773781,RSS Clip Ring Formed,83.000000,1331.000000,Cam???,1000.000000,34.000000,2.120210
48,003663-2-1,NV15327,1773781,RSS Clip Ring Formed,27.000000,1331.000000,Cam???,1000.000000,34.000000,0.689707
57,003663-21-2,NV15327,1773781,RSS Clip Ring Formed,21.000000,1331.000000,Cam???,1000.000000,34.000000,0.536439
60,003663-21-3,NV15327,1773781,RSS Clip Ring Formed,91.000000,1331.000000,Cam???,1000.000000,34.000000,2.324568


In [15]:
so3663_df[so3663_df['PhaseID'].str.contains('Cam')] = 'Clip/Split Ring'

so3663_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3663.csv", columns=so3663_df.columns, index=False, date_format=None)

## 3664

#### Poly Fenestrated and Precept-Poly Screws

In [16]:
so3664_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments2.sql')) as f:
    so3664_query = f.read().split(';')[0].format(SO=3664)

with create_engine(epicorConnectionString).connect() as con:
    so3664_df = pd.read_sql(text(so3664_query), con)

    
# Some Clip Rings are mistakenly labeled as Cams, so just correcting those
so3664_df[so3664_df['PhaseID'].str.contains('Insert')].style.applymap(highlight_yellow, pd.IndexSlice[['JobNum','TranQty','PhaseID']])

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
3,003664-10-1,nv15618,1582023,Precept Poly C.R. Load Ring,40.000000,548.000000,Insert???,500.000000,29.000000,2.116788
6,003664-10-2,NV15618,1582023,Precept Poly C.R. Load Ring,39.000000,548.000000,Insert???,500.000000,29.000000,2.063869
15,003664-11-1,nv15618,1582023,Precept Poly C.R. Load Ring,77.000000,548.000000,Insert???,500.000000,29.000000,4.074818
18,003664-12-1,nv15618,1582023,Precept Poly C.R. Load Ring,27.000000,548.000000,Insert???,500.000000,29.000000,1.428832
22,003664-13-1,nv15618,1582023,Precept Poly C.R. Load Ring,33.000000,548.000000,Insert???,500.000000,29.000000,1.746350
43,003664-16-1,nv15618,1582023,Precept Poly C.R. Load Ring,55.000000,548.000000,Insert???,500.000000,29.000000,2.910584
47,003664-17-1,nv15618,1582023,Precept Poly C.R. Load Ring,30.000000,548.000000,Insert???,500.000000,29.000000,1.587591
51,003664-18-1,nv15618,1582023,Precept Poly C.R. Load Ring,3.000000,548.000000,Insert???,500.000000,29.000000,0.158759
52,003664-18-1,nv15618,1582023,Precept Poly C.R. Load Ring,50.000000,548.000000,Insert???,500.000000,29.000000,2.645985


In [23]:
# Changin these to Load Rings, as they should be 
so3664_df[so3664_df['PhaseID'].str.contains('Insert')] = 'Load Ring'


# Also we have a random reduction assembly job in this order, just because every other job does not have Extensions, we are going to make it homegenous and drop this job
so3664_df = so3664_df[~(so3664_df['JobNum'].str.contains('003664-9-1'))]

In [26]:
so3664_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3664.csv", columns=so3664_df.columns, index=False, date_format=None)

## 3694

#### Reline C-Screws

In [27]:
so3694_query = None

with open(os.getcwd().replace('notebooks','sql\\InventoryAssignments2.sql')) as f:
    so3694_query = f.read().split(';')[0].format(SO=3694)

with create_engine(epicorConnectionString).connect() as con:
    so3694_df = pd.read_sql(text(so3694_query), con)

    
# Just a single row that was labeled as Screw instead of Shank
so3694_df[so3694_df['PhaseID'].str.contains('Screw')].style.applymap(highlight_yellow, pd.IndexSlice[['JobNum','TranQty','PhaseID']])

,JobNum,LotNum,DrawNum,PartDescription,TranQty,QtyCompleted,PhaseID,OrigProdQty,AQLAmount,AQLforTranQty
64,003694-3-1,nv15764,1906XXX,5.0 x 28mm Reline Cervical Polyaxial Screw Shank,28.000000,28.000000,Screw???,25.000000,13.000000,13.000000


In [36]:
so3694_df.loc[64,'PhaseID'] = 'Shank'
so3694_df.to_csv(path_or_buf=os.getcwd().replace('notebooks','csv') + "\\3694.csv", columns=so3694_df.columns, index=False, date_format=None)